# Cas d'usage n°4

## Millésimer un jeu de données

Dans cet exemple, nous allons nous intéresser aux 
[sites pollués gérés par l'ADEME](https://data.ademe.fr/datasets/srd-ademe).

### Constituer le jeu de données

In [1]:
import io
import os
import pandas as pd
from requests_cache import CachedSession

s = CachedSession()

url = "https://data.ademe.fr/data-fair/api/v1/datasets/srd-ademe/lines?size=10000&page=1&format=csv"

r = s.get(url)
df = pd.read_csv(io.BytesIO(r.content), sep=",", dtype=str)
df

,Nom_site,Typologie_intervention_ADEME,Commune,Code_INSEE,Activites_historiques_site,Description_activites,Surface_site_m2,Superficie_bati_m2,Arrete_Prefectoral_Travaux_Office,Numeros_parcelles_cadastrales,Resume_intervention_ADEME,Liens_vers_BASOL_BASIAS,Usage_actuel_site,Region,Departement,CoordXLambert2Etendue,CoordYLambert2Etendue,Etat_operation,fiches_export
0,ARDENITY,Gestion et élimination de déchets,Givet,8190,"Matériaux isolants (Fabrication, Production)","A parti des années 1980, le site a accueilli d...",19700,7000,La mission de l'ADEME était encadrée par arrêt...,000AY34 - 000AY37 - 000AY57 - 000AY64 000AY67 ...,"En 2006, ARDENITY exploite le site concentrant...",http//basol.developpement-durable.gouv.fr/fich...,Friche,Grand-Est,8,777397.3,2574527.8,Terminée,ARDENITY_08190_10112020.pdf
1,ATELIERS DE BADAN,"Gestion et élimination de déchets, Investigati...",Grigny,69096,Ateliers mécaniques,"En 1984, la société des ATELIERS DE BADAN s'es...",550000,12000,NaN,69096000AS102,NaN,NaN,non renseigné,Auvergne-Rhone-Alpes,69,791155.3,2069936.3,Terminée,ATELIERS DE BADAN_69096_14042020.pdf
2,ATLAS INNOVATION,"Gestion et élimination de déchets, Investigati...",Issé,44075,"Plastique (Fabrication, Production, Transforma...",De 1958 à 2003 la société Atlas Plastique exe...,71000,32000,La mission de l'ADEME était encadrée par 2 arr...,000AC162 - 000AC163 - 000AC164 - 000AC210 - 00...,Diverses sociétés ont mené une activité d'endu...,https//basol.developpement-durable.gouv.fr/fic...,non renseigné,Pays-de-la-Loire,44,315310.9,2298072.7,Terminée,ATLAS_INNOVATIONS_44075_08102020.pdf
3,AUREATEX,Gestion et élimination de déchets,Saulce-sur-rhone,8402,"Textile, Teinturerie (production, préparation)",La SAS AUREATEX exerà§ait une activité de la p...,28100,13500,NaN,08402000AB233,NaN,NaN,non renseigné,Auvergne-Rhone-Alpes,26,795167.9,1970313.9,Terminée,AUREATEX_26337_21042020.pdf
4,BAUDOU,"Gestion et élimination de déchets, Investigati...",Les églisottes-et-Chalaures,33154,"Plastique (Fabrication, Production, Transforma...",Ancien site industriel de fabrication de chaus...,45200,8000,La mission de l'ADEME était encadrée par arrêt...,000AB109 - 000AB389 - 000AB390 - 000AB391 - 00...,Ancien site industriel de fabrication de chaus...,NaN,non renseigné,Nouvelle-Aquitaine,33,413060.6,2014069.8,Terminée,BAUDOU_33154_06102020.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,WIPELEC,"Investigations environnementales, Travaux, Dép...",Romainville,93063,"Traitement de surface, Revêtements, Applicatio...",Une activité de traitement de surface a été ex...,3500,1150,La mission de l'ADEME était encadrée par arrêt...,000AM181 à 183 000AM185 - 000AM224 - 000AM225...,Une activité de traitement de surface a été ex...,NaN,Non renseigné,Ile-de-France,93,607798.1,2430480.9,En cours,NaN
415,WONDER (Collège Ariane et habitation en proxim...,"Travaux, Dépollution et Maîtrise des transfert...",Vernon,27681,Fabrication de piles,Collège construit sur l'emprise d'anciens terr...,non renseigné,non renseigné,La mission de l'ADEME est encadrée par arrêté ...,NaN,College construit sur l'emprise d'anciens terr...,NaN,non renseigné,Normandie,27,536807.61,2455540.21,En cours,NaN
416,YERAMEX,"Gestion et élimination de déchets, Investigati...",Le Saulcy,88444,"Plastique (Fabrication, Production, Transforma...",L'usine YERAMEX permettait la fabrication de t...,24400,6000,La mission de l'ADEME était encadrée par arrêt...,000A2283 à 0002294 - 000A679 - 000A899 - 000A904,L'usine YERAMEX permettait la fabrication de t...,NaN,Non renseigné,Grand-Est,88,949347.5,2390410.7,En cours,NaN
417,ZAC DES AIGUILLES,Investigations environnementales,Gignac-le-Nerthe,13043,Dépôt (déchets divers),"Début 2019, dans le cadre de l'élaboration des...",9150,non renseigné,La mission de l'ADEME est encadrée par arrêté ...,NaN,"Début 2019, dans le cadre de l'élaboration des...",NaN,Non renseigné,PACA,13,832563.94,1823882.76,En cours,NaN


### Corriger le jeu de données

Certains codes INSEE ont été mal formatés dans le fichier source : 
malgré la précaution de spécifier un type "str", il manque des 0 en tête de
code commune pour les neuf premiers départements français.

In [2]:
ix = df[df.Code_INSEE.str.len() == 4].index
df.loc[ix, "Code_INSEE"].to_frame()

,Code_INSEE
0,8190
3,8402
11,2173
23,3185
27,9157
35,7013
55,3229
90,3315
104,1053
133,6054


Commençons donc par corriger ces erreurs de format.

In [3]:
df["Code_INSEE"] = df["Code_INSEE"].str.zfill(5)
df.loc[ix, "Code_INSEE"].to_frame()

,Code_INSEE
0,08190
3,08402
11,02173
23,03185
27,09157
35,07013
55,03229
90,03315
104,01053
133,06054


### Millésimer le jeu de données

Configuration de l'API INSEE

In [4]:
from french_cities import set_vintage
os.environ["insee_key"] = "********************"
os.environ["insee_secret"] = "********************"

Stocker les codes initiaux pour effectuer une comparaison ultérieure

In [5]:
init = df["Code_INSEE"].copy()
init.name = "initial"

Utiliser french-cities

In [6]:
df = set_vintage(df, 2024, "Code_INSEE")

Comparer les résultats aux données initiales

In [7]:
new = df["Code_INSEE"].copy()
new.name = "final"

Lister les modifications effectuées :

In [8]:
(init == new).all()

False

In [9]:
ix = init[init != new].index
init.to_frame().join(new).loc[ix]

,initial,final
155,38014,38297
177,27270,27070
257,91182,91228
287,74093,74010
288,79041,79005
343,73002,73212
365,73127,73236
406,33091,33268


Le jeu de donnée contenait 9 codes communes obsolètes et qui ont été projetés dans le millésime courant.